<a href="https://colab.research.google.com/github/User8549/MyRep/blob/main/%D0%91%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

Пример запуска

In [ ]:
%%cu
#include <iostream>

__global__ void gpu_f()
{
    printf("hello \n");
}

int main()
{
    cudaDeviceProp dev{};
    cudaGetDeviceProperties(&dev, 0);
    std::cout << "Multiprocessors in device = " << dev.multiProcessorCount << std::endl;
    gpu_f<<<dev.multiProcessorCount, 10>>>();
    cudaDeviceSynchronize();
    return 0;
}

Задание 1
написать программу которая с видеокарты пишет сообщение вида "Hello from a thread b block", где a и b номера нити и блока соответственно

In [ ]:
%%cu
#include <iostream>

__global__ void gpu_f()
{
    printf("Hello from block: %d, thread: %d\n",blockIdx.x,threadIdx.x);
}

int main()
{
    gpu_f<<<5, 3>>>();
    cudaDeviceSynchronize();
    return 0;
}

заданиие 2: 
умножение матриц

In [ ]:
%%cu
#include <iostream>

__global__ void dev(int* mass1, int* mass2, int* mass3, int n, int m)
{
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    if(x < (n * n)){
        int mm = (int)(x / n);
        int nn = (int)(x % n);
        mass3[x] = 0;
        for(auto j = 0; j < m; j++){
            mass3[x] += mass1[j + m * mm] * mass2[j * n + nn];  
        }
        printf ("x = %d, dev = %d \n", x,mass3[x]);
    }
}

void create(int* mass, int n, int m)
{
    for(auto i = 0; i < n; i++){
        for(auto j = 0; j < m; j++)
        {
            mass[i * m + j] = rand() % 21 - 10;
        }
    }
}
void show(int* mass, int n, int m)
{
    for(auto i = 0; i < n; i++)
    {
        for (auto j = 0; j < m; j++)
        {
            std::cout << mass[i * m + j] << " ";
        }
        std::cout << "\n";
    }
    std::cout << "\n";
}

int main()
{
    int *a, *b, *c;

    int n = 2; int x = 4; //строки
    int m = 4; int y = 2; //столбцы
    
    if (m != x) {
      printf("ERROR SIZE");
      return 0;
    }
 
    cudaMallocManaged(&a,n * m * sizeof(int));
    cudaMallocManaged(&b,x * y * sizeof(int));
    cudaMallocManaged(&c,n * y * sizeof(int));
 
    create(a, n, m);
    create(b, x, y);
 
    show(a, n, m);
    show(b, x, y);
 
    dev<<<n, m>>>(a, b, c, n, m);
    cudaDeviceSynchronize();
    
    std::cout << "\n";
    show(c, n, y);
    cudaFree (a);
    cudaFree (b);
    cudaFree (c);
    return 0;
}


Задание 3: параллельно отсортировать одномерный массив

In [160]:
%%cu
#include <iostream>

__global__ void sort(int *arr, int n)
{
    auto x = threadIdx.x + blockIdx.x * blockDim.x; //нить

    for (int i = 0; i < n / 2; i++){
        if (x % 2 == 0 && (x + 1) < n && x < n){ // чет
            if(arr[x] > arr[x + 1]){
                int temp = arr[x];
                arr[x] = arr[x+1];
                arr[x+1] = temp;
            }
        }
        __syncthreads(); //потоки
        __threadfence(); //блоки
        if(x % 2 != 0 && (x + 1) < n && x < n) { // нечет
            if(arr[x] > arr[x + 1]){
                int temp = arr[x];
                arr[x] = arr[x+1];
                arr[x+1] = temp;
            }
        }
        __syncthreads(); //потоки
        __threadfence(); //блоки
    }
}

void create(int* mass, int n)
{
    for(auto i = 0; i < n; i++)
            mass[i] = rand()%50;
}

void show(int* mass, int n)
{
    for (auto i = 0; i < n; i++)
    {
        std::cout << mass[i] << " ";
    }
    std::cout << "\n";
}

int main()
{
    int *a;
    int n = 15;
    cudaMallocManaged(&a,n * sizeof(int));
    create(a, n);
    show(a, n);
    sort<<<5, 5>>>(a, n);
    cudaDeviceSynchronize();
    show(a, n);
    return 0;
}


33 36 27 15 43 35 36 42 49 21 12 27 40 9 13 
9 12 13 15 21 27 27 33 35 36 36 40 42 43 49 

